In [1]:
import pandas as pd
import numpy as np  
from cobra.core.gene import parse_gpr, ast2str, GPR
import cobra
from helper.eval_gpr import evaluate_gpr



# Read Graph From File

In [2]:
all_nodes = pd.read_csv('./network_data/removed_metabolitic_and_mirna_nodes_without_ppi.csv')
all_edges = pd.read_csv('./network_data/removed_metabolitic_and_mirna_edges_without_ppi.csv')

all_nodes = all_nodes.iloc[:,4:]
all_edges = all_edges.iloc[:,4:]

# Read Metabolomics and Transcriptomics Data

In [3]:
data = pd.read_csv('./datasets/cancer atlas/data/MasterMapping_MetImmune_03_16_2022_release.csv')

In [4]:
filtered_data = data[data['Identifier'].str.contains('BRCA_Terunuma')]
# filtered_data = filtered_data.iloc[40:60]
# filtered_data

In [5]:
metabolomic_data = pd.read_excel('./datasets/cancer atlas/data/metabolomics_original/BRCA1.xlsx')


/Users/utku/anaconda3/envs/metaboliticsenv/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [6]:
metabolomic_data = metabolomic_data.iloc[350:]
# metabolomic_data

In [7]:
column_metabolomic_data = metabolomic_data.columns
column_metabolomic_data = column_metabolomic_data[10:]
# filtered_data['MetabID'].values

In [8]:
common_elements = np.intersect1d(column_metabolomic_data.astype(str).values, filtered_data['MetabID'].astype(str).values)
# common_elements

In [9]:
normal_metabId = filtered_data[filtered_data['TN'] == 'Normal']['MetabID'].values
tumor_metabId = filtered_data[filtered_data['TN'] == 'Tumor']['MetabID'].values
normal_rnaId = filtered_data[filtered_data['TN'] == 'Normal']['RNAID'].values
tumor_rnaId = filtered_data[filtered_data['TN'] == 'Tumor']['RNAID'].values

# tumor_metabId

In [10]:
metabId_and_rnaId_mapping = filtered_data[['MetabID', 'RNAID']]
metabId_and_rnaId_mapping.to_csv('./datasets/cancer atlas/metabId_and_rnaId_mapping.csv')

In [11]:
transcriptomic_data = pd.read_csv('./datasets/cancer atlas/data/transcriptomics_processed/GSE37751.hugene10st.gene_symbol.csv')

In [12]:
healty_average_data_cancer_atlas_gene = pd.DataFrame({
    'rna_id': transcriptomic_data['gene_symbol'],
    'average_score': transcriptomic_data[normal_rnaId].mean(axis=1)
})
# healty_average_data_cancer_atlas_gene

In [13]:
healty_average_data_cancer_atlas_gene.to_csv('./datasets/cancer atlas/healty_average_data_brca1_for_all.csv')


In [14]:
healty_average_data_cancer_atlas_gene = pd.read_csv('./datasets/cancer atlas/healty_average_data_brca1_for_all.csv')

In [15]:
def find_gene_reaction_rule_path_from_mrna(start_node, all_edges):
    transcribed_to_edges = all_edges[(all_edges['interaction'] == 'transcribed_to') & (all_edges['source'] == start_node)]
    def find_next_edge(current_target, edges_df):
        next_edges = edges_df[edges_df['source'] == current_target]
        return next_edges
    all_path = []

    for index, row in transcribed_to_edges.iterrows():
        diffused_value = all_nodes.loc[all_nodes['id'] == row['source'], 'experimented_value'].values[0] 
        
        starting_point = row['source']
        current_target = row
        transcription_path = [] 
        if not pd.isna(all_nodes[all_nodes['id'] == current_target['target']]['experimented_value'].values[0]):
            diffused_value = all_nodes.loc[all_nodes['id'] == current_target['target'], 'experimented_value'].values[0]
        transcription_path.append((row['source'], row['target'],  row['interaction'], diffused_value))
        while True:
            next_edges = find_next_edge(current_target['target'], all_edges)
            if not next_edges.empty and next_edges[(next_edges['interaction'] == 'consumed') | (next_edges['interaction'] == 'produced')].empty:
                next_edge = next_edges.sample(n=1).iloc[0]
                current_target = next_edge
                if current_target['interaction'] == 'Repression':
                    diffused_value = -1*diffused_value 
                if not pd.isna(all_nodes[all_nodes['id'] == current_target['target']]['experimented_value'].values[0]):
                    diffused_value = all_nodes.loc[all_nodes['id'] == current_target['target'], 'experimented_value'].values[0]
                elif not pd.isna(all_nodes[all_nodes['id'] == current_target['target']]['diffused_value'].values[0]) and all_nodes.loc[all_nodes['id'] == current_target['target'], 'diffused_value'].values[0] > diffused_value:
                    diffused_value = all_nodes.loc[all_nodes['id'] == current_target['target'], 'diffused_value'].values[0]
                all_nodes.loc[all_nodes['id'] == current_target['target'], 'diffused_value'] = diffused_value
                transcription_path.append((next_edge['source'], current_target['target'],  next_edge['interaction'], diffused_value))
            else:
                break  # Bir sonraki edge bulunamazsa döngüden çık
        all_path.append(transcription_path)
    return all_path

In [16]:
from joblib import Parallel, delayed

def find_gene_reaction_rule_path_from_mrna_parallel(start_node, all_edges, all_nodes):
    transcribed_to_edges = all_edges[(all_edges['interaction'] == 'transcribed_to') & (all_edges['source'] == start_node)]

    def find_next_edge(current_target, edges_df):
        next_edges = edges_df[edges_df['source'] == current_target]
        return next_edges

    def process_path(row, all_nodes):
        diffused_value = all_nodes.loc[all_nodes['id'] == row['source'], 'experimented_value'].values[0] 
        starting_point = row['source']
        current_target = row
        transcription_path = [] 
        if not pd.isna(all_nodes[all_nodes['id'] == current_target['target']]['experimented_value'].values[0]):
            diffused_value = all_nodes.loc[all_nodes['id'] == current_target['target'], 'experimented_value'].values[0]
        transcription_path.append((row['source'], row['target'],  row['interaction'], diffused_value))
        while True:
            next_edges = find_next_edge(current_target['target'], all_edges)
            if not next_edges.empty and next_edges[(next_edges['interaction'] == 'consumed') | (next_edges['interaction'] == 'produced')].empty:
                next_edge = next_edges.sample(n=1).iloc[0]
                current_target = next_edge
                if current_target['interaction'] == 'Repression':
                    diffused_value = -1 * diffused_value 
                if not pd.isna(all_nodes[all_nodes['id'] == current_target['target']]['experimented_value'].values[0]):
                    diffused_value = all_nodes.loc[all_nodes['id'] == current_target['target'], 'experimented_value'].values[0]
                elif not pd.isna(all_nodes[all_nodes['id'] == current_target['target']]['diffused_value'].values[0]) and all_nodes.loc[all_nodes['id'] == current_target['target'], 'diffused_value'].values[0] > diffused_value:
                    diffused_value = all_nodes.loc[all_nodes['id'] == current_target['target'], 'diffused_value'].values[0]
                all_nodes.loc[all_nodes['id'] == current_target['target'], 'diffused_value'] = diffused_value
                transcription_path.append((next_edge['source'], current_target['target'],  next_edge['interaction'], diffused_value))
            else:
                break  # Bir sonraki edge bulunamazsa döngüden çık
        return transcription_path

    all_path = Parallel(n_jobs=-1)(delayed(process_path)(row, all_nodes) for index, row in transcribed_to_edges.iterrows())
    return all_path


# Fold Change

In [17]:
indexNumber = 0
# for column in tumor_rnaId:
for column in np.concatenate([normal_rnaId, tumor_rnaId]):
    indexNumber += 1
    # print(indexNumber, column)
    # transcriptomic_data.loc[:, column] = (transcriptomic_data.loc[:, column] / healty_average_data_cancer_atlas_gene['average_score'].values) - 1
    transcriptomic_data[column] = (transcriptomic_data[column] / healty_average_data_cancer_atlas_gene['average_score']) - 1

In [18]:
chosen_columns = np.concatenate([['gene_symbol'],normal_rnaId, tumor_rnaId])
transcriptomic_data[chosen_columns]

,gene_symbol,GSM926965.CEL.gz,GSM926966.CEL.gz,GSM926967.CEL.gz,GSM926968.CEL.gz,GSM926969.CEL.gz,GSM926970.CEL.gz,GSM926971.CEL.gz,GSM926972.CEL.gz,GSM926973.CEL.gz,...,GSM927061.CEL.gz,GSM927062.CEL.gz,GSM927063.CEL.gz,GSM927064.CEL.gz,GSM927065.CEL.gz,GSM927066.CEL.gz,GSM927067.CEL.gz,GSM927068.CEL.gz,GSM927069.CEL.gz,GSM927070.CEL.gz
0,NAT2,0.003940,-0.011222,-0.013250,0.338757,-0.048422,-0.020895,0.052870,-0.034900,0.004299,...,-0.042231,-0.013400,-0.018454,-0.024868,-0.039627,0.025126,0.078509,0.007241,-0.062600,0.014942
1,ADA,0.128942,-0.045482,0.036866,-0.055464,-0.003240,-0.048033,-0.094063,-0.018592,-0.023622,...,0.104954,-0.104370,-0.016936,0.160371,-0.064876,-0.002973,-0.086799,0.068904,0.072417,-0.045665
2,CDH2,0.003913,0.016386,-0.022204,-0.079019,0.033736,-0.036220,0.022403,-0.044511,-0.006223,...,-0.021968,0.000825,0.040168,0.023362,-0.039888,0.132299,0.093649,0.098130,-0.004901,-0.018123
3,AKT3,-0.077588,-0.059214,0.039886,-0.042198,0.028722,-0.005944,-0.013116,-0.021506,0.050432,...,-0.181193,-0.046795,-0.048431,-0.156806,0.023243,0.094044,0.013990,-0.078284,0.002990,-0.175338
4,RNA28SN5,0.008961,0.007504,0.001666,-0.001959,0.006089,0.000268,0.005249,-0.004361,0.003433,...,-0.008224,-0.004137,-0.005176,-0.010761,-0.001752,-0.006927,-0.002899,0.000731,-0.001837,0.009965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20027,PTBP3,-0.049137,0.020266,-0.049180,0.019948,-0.005960,0.022330,0.017129,0.009185,0.011740,...,0.041989,0.014661,-0.004942,0.084647,0.006234,0.019576,0.047333,0.071226,0.067452,0.040794
20028,KCNE2,-0.013777,-0.073143,0.026489,0.026427,0.003179,0.010287,-0.023419,-0.010800,-0.053371,...,-0.021709,-0.001722,-0.013074,-0.101265,-0.019110,-0.008199,-0.086298,-0.011582,-0.054395,-0.002904
20029,DGCR2,-0.017257,-0.002335,0.019033,0.021163,-0.001849,0.015312,0.003656,0.002554,0.006760,...,0.054944,0.046995,0.014209,0.027027,0.016582,-0.007670,0.026997,0.012373,-0.003252,0.014761
20030,CASP8AP2,-0.020176,0.004796,-0.013570,0.002014,-0.002998,0.016918,0.008278,0.012320,-0.018187,...,-0.006074,-0.018379,-0.047819,0.071196,-0.028483,-0.006759,-0.016508,0.033869,0.078250,-0.041367


In [19]:
transcriptomic_data = transcriptomic_data[chosen_columns]
transcriptomic_data.to_csv('./datasets/cancer atlas/fold_change_transcriptomics_for_all.csv')

In [20]:
transcriptomic_data = pd.read_csv('./datasets/cancer atlas/fold_change_transcriptomics_for_all.csv')
transcriptomic_data_test = pd.read_csv('./datasets/cancer atlas/test_transcriptomics.csv')

In [21]:
transcriptomic_data = transcriptomic_data.iloc[:, 1:]
transcriptomic_data
transcriptomic_data_test = transcriptomic_data_test.iloc[:, 1:]
transcriptomic_data_test

,gene_symbol,GSM926965.CEL.gz,GSM926966.CEL.gz,GSM926967.CEL.gz,GSM926968.CEL.gz,GSM926969.CEL.gz,GSM926970.CEL.gz,GSM926971.CEL.gz,GSM926972.CEL.gz,GSM926973.CEL.gz,...,GSM927061.CEL.gz,GSM927062.CEL.gz,GSM927063.CEL.gz,GSM927064.CEL.gz,GSM927065.CEL.gz,GSM927066.CEL.gz,GSM927067.CEL.gz,GSM927068.CEL.gz,GSM927069.CEL.gz,GSM927070.CEL.gz
0,NAT2,0.003940,-0.011222,-0.013250,0.338757,-0.048422,-0.020895,0.052870,-0.034900,0.004299,...,-1.226745,-1.225290,-1.225545,-1.225869,-1.226613,-1.223346,-1.220653,-1.224249,-1.227772,-1.223860
1,ADA,0.128942,-0.045482,0.036866,-0.055464,-0.003240,-0.048033,-0.094063,-0.018592,-0.023622,...,-1.133628,-1.137474,-1.135867,-1.132609,-1.136748,-1.135611,-1.137151,-1.134290,-1.134225,-1.136395
2,CDH2,0.003913,0.016386,-0.022204,-0.079019,0.033736,-0.036220,0.022403,-0.044511,-0.006223,...,-1.165737,-1.165116,-1.164043,-1.164501,-1.166226,-1.161530,-1.162584,-1.162462,-1.165272,-1.165632
3,AKT3,-0.077588,-0.059214,0.039886,-0.042198,0.028722,-0.005944,-0.013116,-0.021506,0.050432,...,-1.123285,-1.121328,-1.121352,-1.122930,-1.120309,-1.119278,-1.120443,-1.121787,-1.120604,-1.123199
4,RNA28SN5,0.008961,0.007504,0.001666,-0.001959,0.006089,0.000268,0.005249,-0.004361,0.003433,...,-1.073286,-1.073264,-1.073269,-1.073299,-1.073251,-1.073279,-1.073257,-1.073238,-1.073251,-1.073188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,PSORS1C3,-0.043901,-0.050438,0.011423,-0.009277,-0.007961,0.077814,-0.055331,0.008650,0.002469,...,-1.215953,-1.211921,-1.213924,-1.217265,-1.214308,-1.214912,-1.215212,-1.216887,-1.217955,-1.221418
148,ZNF663P,0.014035,-0.026095,0.012661,-0.043568,-0.029750,0.014665,-0.022488,0.042609,-0.029826,...,-1.196239,-1.199090,-1.200540,-1.200448,-1.198447,-1.199356,-1.198068,-1.200989,-1.199181,-1.197292
149,SYCE1L,-0.024321,-0.019086,0.012560,0.038397,0.001612,-0.015883,-0.039025,0.003762,-0.037517,...,-1.129444,-1.128898,-1.129190,-1.129680,-1.128766,-1.129693,-1.129319,-1.128563,-1.128730,-1.130196
150,C6orf99,0.127673,0.045013,-0.052762,0.041550,0.059976,0.006419,-0.020195,-0.009531,-0.056148,...,-1.172317,-1.172351,-1.170574,-1.171420,-1.171507,-1.171562,-1.172770,-1.169440,-1.175036,-1.170787


In [22]:
def update_experimented_value(row, all_nodes, column):
    gene_symbol = row['gene_symbol']
    transcript_id = gene_symbol + '_transcript'
    if not all_nodes[all_nodes['id'] == transcript_id].empty:
        all_nodes.loc[all_nodes['id'] == transcript_id, 'experimented_value'] = row[column]


for column in np.concatenate([tumor_rnaId, normal_rnaId]):
    transcriptomic_data.apply(lambda row: update_experimented_value(row, all_nodes, column), axis=1)
    all_nodes.to_csv(f"updated_all_nodes/all_nodes_{column}.csv", index=False)



In [ ]:
from joblib import Parallel, delayed
import os

def find_gene_reaction_rule_path_from_mrna_parallel(start_node, all_edges, all_nodes):
    transcribed_to_edges = all_edges[(all_edges['interaction'] == 'transcribed_to') & (all_edges['source'] == start_node)]

    def find_next_edge(current_target, edges_df):
        next_edges = edges_df[edges_df['source'] == current_target]
        return next_edges

    def process_path(row, all_nodes):
        diffused_value = all_nodes.loc[all_nodes['id'] == row['source'], 'experimented_value'].values[0] 
        starting_point = row['source']
        current_target = row
        result_path = []  # Boş bir liste oluştur

        if not pd.isna(all_nodes[all_nodes['id'] == current_target['target']]['experimented_value'].values[0]):
            diffused_value = all_nodes.loc[all_nodes['id'] == current_target['target'], 'experimented_value'].values[0]
        while True:
            next_edges = find_next_edge(current_target['target'], all_edges)
            if not next_edges.empty and next_edges[(next_edges['interaction'] == 'consumed') | (next_edges['interaction'] == 'produced')].empty:
                next_edge = next_edges.sample(n=1).iloc[0]
                current_target = next_edge
                if current_target['interaction'] == 'Repression':
                    diffused_value = -1 * diffused_value 
                if not pd.isna(all_nodes[all_nodes['id'] == current_target['target']]['experimented_value'].values[0]):
                    diffused_value = all_nodes.loc[all_nodes['id'] == current_target['target'], 'experimented_value'].values[0]
                elif not pd.isna(all_nodes[all_nodes['id'] == current_target['target']]['diffused_value'].values[0]) and all_nodes.loc[all_nodes['id'] == current_target['target'], 'diffused_value'].values[0] > diffused_value:
                    diffused_value = all_nodes.loc[all_nodes['id'] == current_target['target'], 'diffused_value'].values[0]
                all_nodes.loc[all_nodes['id'] == current_target['target'], 'diffused_value'] = diffused_value
                result_path.append((next_edge['source'], current_target['target'],  next_edge['interaction'], diffused_value))
            else:
                break  
        return result_path  # Döngü tamamlandığında result_path'i döndür

    Parallel(n_jobs=-1)(
        delayed(process_path)(row, all_nodes) for index, row in transcribed_to_edges.iterrows()
    )


def process_column(column, transcriptomic_data, all_edges):
    if not os.path.exists(f"gpr_rule_all_nodes/all_nodes_{column}.csv"):

        all_nodes = pd.read_csv(f"updated_all_nodes/all_nodes_{column}.csv")
        column_data = transcriptomic_data[['gene_symbol', column]]
        results = []

        def process_row(index, row, all_nodes, all_edges):
            row_results = []
            print(row['gene_symbol'])
            if not all_nodes[all_nodes['id'] == row['gene_symbol'] + '_transcript'].empty:
                result_path = find_gene_reaction_rule_path_from_mrna_parallel(row['gene_symbol'], all_edges, all_nodes)
        Parallel(n_jobs=-1)(
            delayed(process_row)(index, row, all_nodes, all_edges)
            for index, row in column_data.iterrows()
        )

        final_results = [item for sublist in results for item in sublist]
        all_nodes.to_csv(f"gpr_rule_all_nodes/all_nodes_{column}.csv", index=False)

Parallel(n_jobs=-1)(
    delayed(process_column)(column, transcriptomic_data, all_edges)
    for column in tumor_rnaId
)


# Load Recon3D model

In [ ]:
model = cobra.io.load_json_model("./datasets/network_datasets/Recon3D.json")

In [ ]:
import json

with open('./datasets/network_datasets/Recon3D.json', 'r') as file:
    recon3d = json.load(file)


In [ ]:
loaded_reactions = recon3d['reactions']

In [ ]:
number = 0
for column in tumor_rnaId:
    number += 1
    
    all_nodes = pd.read_csv(f"gpr_rule_all_nodes/all_nodes_{column}.csv")
    all_edges = pd.read_csv('./network_data/removed_metabolitic_and_mirna_edges_without_ppi.csv')
    column_data = (transcriptomic_data[['gene_symbol', column]])

    with open('./datasets/network_datasets/Recon3D.json', 'r') as file:
        recon3d = json.load(file)

    loaded_reactions = recon3d['reactions']

    result_array = []
    for index, reaction_row in all_nodes[all_nodes['group'] == 'reaction'].iterrows():
        reaction = model.reactions.get_by_id(reaction_row.id)

        _, frozenset = parse_gpr(reaction.gene_reaction_rule)
        gene_values = {}
        proteins = []
        for gene in frozenset:
            protein_node_name = str(model.genes.get_by_id(str(gene)).name) + '_protein'
            proteins.append(protein_node_name)
        if(len(all_nodes[(all_nodes['id'].isin(proteins)) & (pd.notna(all_nodes['diffused_value']))]) > 0):
            gene_values = {}
            for gene in frozenset: 
                gene_values[gene] = all_nodes.loc[all_nodes['id'] == str(model.genes.get_by_id(str(gene)).name) + '_protein', 'diffused_value'].values[0]
            nan_genes = {key for key, value in gene_values.items() if np.isnan(value)}
            result = evaluate_gpr(reaction.gene_reaction_rule, knockouts=nan_genes, gene_values=gene_values)
            if(result[0] < 0.0):
                desired_reaction = next((reaction for reaction in loaded_reactions if reaction['id'] == 'INSTt4'), None)
                if desired_reaction:
                    next((reaction for reaction in loaded_reactions if reaction['id'] == reaction_row.id), None)['lower_bound'] = next((reaction for reaction in loaded_reactions if reaction['id'] == str(reaction_row.id)), None)['lower_bound']*result[0]
                    next((reaction for reaction in loaded_reactions if reaction['id'] == reaction_row.id), None)['upper_bound'] = next((reaction for reaction in loaded_reactions if reaction['id'] == str(reaction_row.id)), None)['upper_bound']*result[0]

            elif(result[0] > 0.0):
                desired_reaction = next((reaction for reaction in loaded_reactions if reaction['id'] == 'INSTt4'), None)
                if desired_reaction:
                    next((reaction for reaction in loaded_reactions if reaction['id'] == reaction_row.id), None)['lower_bound'] = next((reaction for reaction in loaded_reactions if reaction['id'] == str(reaction_row.id)), None)['lower_bound']*result[0]
        result_array.append(result)

    print("reaction constraints are changed")

    with open('./datasets/network_datasets/Recon3D.json', 'r') as file:
        new_recond3d = json.load(file)

    new_recond3d['reactions'] = loaded_reactions

    with open('./updated_recon3d_samples/updated_Recon3D_' + str(number) + '.json', 'w') as file:
        json.dump(new_recond3d, file, indent=2)
    




In [124]:
import joblib
from joblib import delayed

def process_column(column):
    # if not os.path.exists('./updated_recon3d_samples/updated_Recon3D_' + str(number) + '.json'):

    all_nodes = pd.read_csv(f"gpr_rule_all_nodes/all_nodes_{column}.csv")
    all_edges = pd.read_csv('./network_data/removed_metabolitic_and_mirna_edges_without_ppi.csv')
    column_data = (transcriptomic_data[['gene_symbol', column]])

    with open('./datasets/network_datasets/Recon3D.json', 'r') as file:
        recon3d = json.load(file)

    loaded_reactions = recon3d['reactions']

    result_array = []
    for index, reaction_row in all_nodes[all_nodes['group'] == 'reaction'].iterrows():
        reaction = model.reactions.get_by_id(reaction_row.id)

        _, frozenset = parse_gpr(reaction.gene_reaction_rule)
        gene_values = {}
        proteins = []
        for gene in frozenset:
            protein_node_name = str(model.genes.get_by_id(str(gene)).name) + '_protein'
            proteins.append(protein_node_name)
        if(len(all_nodes[(all_nodes['id'].isin(proteins)) & (pd.notna(all_nodes['diffused_value']))]) > 0):
            gene_values = {}
            for gene in frozenset: 
                gene_values[gene] = all_nodes.loc[all_nodes['id'] == str(model.genes.get_by_id(str(gene)).name) + '_protein', 'diffused_value'].values[0]
            nan_genes = {key for key, value in gene_values.items() if np.isnan(value)}
            result = evaluate_gpr(reaction.gene_reaction_rule, knockouts=nan_genes, gene_values=gene_values)
            if(result[0] < 0.0):
                desired_reaction = next((reaction for reaction in loaded_reactions if reaction['id'] == 'INSTt4'), None)
                if desired_reaction:
                    next((reaction for reaction in loaded_reactions if reaction['id'] == reaction_row.id), None)['lower_bound'] = next((reaction for reaction in loaded_reactions if reaction['id'] == str(reaction_row.id)), None)['lower_bound']*1.5 #result[0]
                    next((reaction for reaction in loaded_reactions if reaction['id'] == reaction_row.id), None)['upper_bound'] = next((reaction for reaction in loaded_reactions if reaction['id'] == str(reaction_row.id)), None)['upper_bound']*0.5

            elif(result[0] > 0.0):
                desired_reaction = next((reaction for reaction in loaded_reactions if reaction['id'] == 'INSTt4'), None)
                if desired_reaction:
                    next((reaction for reaction in loaded_reactions if reaction['id'] == reaction_row.id), None)['lower_bound'] = next((reaction for reaction in loaded_reactions if reaction['id'] == str(reaction_row.id)), None)['lower_bound']*0.5 #result[0]
        result_array.append(result)

    print(f"Processing for column {column} is done")

    with open('./datasets/network_datasets/Recon3D.json', 'r') as file:
        new_recond3d = json.load(file)

    new_recond3d['reactions'] = loaded_reactions

    with open('./updated_recon3d_samples/updated_Recon3D_' + str(column) + '.json', 'w') as file:
        json.dump(new_recond3d, file, indent=2)

joblib.Parallel(n_jobs=-1)(delayed(process_column)(column) for column in tumor_rnaId)


Processing for column GSM927012.CEL.gz is done
Processing for column GSM927051.CEL.gz is done
Processing for column GSM927013.CEL.gz is done
Processing for column GSM927014.CEL.gz is done
Processing for column GSM927010.CEL.gz is done
Processing for column GSM927015.CEL.gz is done
Processing for column GSM927016.CEL.gz is done
Processing for column GSM927011.CEL.gz is done
Processing for column GSM927019.CEL.gz is done
Processing for column GSM927017.CEL.gz is done
Processing for column GSM927018.CEL.gz is done
Processing for column GSM927020.CEL.gz is done
Processing for column GSM927021.CEL.gz is done
Processing for column GSM927024.CEL.gz is done
Processing for column GSM927023.CEL.gz is done
Processing for column GSM927022.CEL.gz is done
Processing for column GSM927025.CEL.gz is done
Processing for column GSM927027.CEL.gz is done
Processing for column GSM927026.CEL.gz is done
Processing for column GSM927028.CEL.gz is done
Processing for column GSM927032.CEL.gz is done
Processing fo

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [55]:
indexNumber = 0
for column in tumor_rnaId:
    index
    print(column)

GSM927051.CEL.gz
GSM927010.CEL.gz
GSM927011.CEL.gz
GSM927012.CEL.gz
GSM927013.CEL.gz
GSM927014.CEL.gz
GSM927015.CEL.gz
GSM927016.CEL.gz
GSM927017.CEL.gz
GSM927018.CEL.gz
GSM927019.CEL.gz
GSM927020.CEL.gz
GSM927021.CEL.gz
GSM927022.CEL.gz
GSM927023.CEL.gz
GSM927024.CEL.gz
GSM927025.CEL.gz
GSM927026.CEL.gz
GSM927027.CEL.gz
GSM927028.CEL.gz
GSM927029.CEL.gz
GSM927030.CEL.gz
GSM927031.CEL.gz
GSM927032.CEL.gz
GSM927033.CEL.gz
GSM927034.CEL.gz
GSM927035.CEL.gz
GSM927036.CEL.gz
GSM927037.CEL.gz
GSM927038.CEL.gz
GSM927039.CEL.gz
GSM927040.CEL.gz
GSM927041.CEL.gz
GSM927042.CEL.gz
GSM927043.CEL.gz
GSM927044.CEL.gz
GSM927045.CEL.gz
GSM927046.CEL.gz
GSM927047.CEL.gz
GSM927048.CEL.gz
GSM927049.CEL.gz
GSM927050.CEL.gz
GSM927052.CEL.gz
GSM927053.CEL.gz
GSM927054.CEL.gz
GSM927055.CEL.gz
GSM927056.CEL.gz
GSM927057.CEL.gz
GSM927058.CEL.gz
GSM927059.CEL.gz
GSM927060.CEL.gz
GSM927061.CEL.gz
GSM927062.CEL.gz
GSM927063.CEL.gz
GSM927064.CEL.gz
GSM927065.CEL.gz
GSM927066.CEL.gz
GSM927067.CEL.gz
GSM927068.CEL.

# Processed Metabolomics Data

In [56]:
metabolomic_data = pd.read_excel('./datasets/cancer atlas/metabolomics_original/BRCA1.xlsx')

metabolomic_data = metabolomic_data.iloc[17:350]


/Users/utku/anaconda3/envs/metaboliticsenv/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [57]:
column_metabolomic_data = metabolomic_data.columns
column_metabolomic_data = column_metabolomic_data[10:]


In [58]:
column_metabolomic_data

Index([499527, 499529, 499531, 499533, 499535, 499537, 499539, 499541, 499543,
       499545,
       ...
       499771, 499773, 499775, 499777, 499779, 499781, 499783, 499785, 499787,
       499789],
      dtype='object', length=132)

In [78]:
import numpy as np
common_elements = np.intersect1d(column_metabolomic_data.astype(int).values, filtered_data['MetabID'].astype(int).values)
(common_elements)

array([499527, 499529, 499533, 499535, 499537, 499539, 499541, 499543,
       499545, 499547, 499551, 499553, 499557, 499559, 499561, 499563,
       499565, 499567, 499573, 499575, 499577, 499581, 499583, 499585,
       499587, 499589, 499591, 499593, 499599, 499601, 499603, 499605,
       499607, 499611, 499613, 499617, 499623, 499625, 499627, 499629,
       499631, 499635, 499637, 499639, 499641, 499643, 499645, 499653,
       499657, 499659, 499661, 499663, 499665, 499667, 499669, 499671,
       499673, 499675, 499677, 499679, 499683, 499685, 499689, 499691,
       499693, 499695, 499697, 499699, 499701, 499703, 499705, 499707,
       499709, 499711, 499713, 499715, 499717, 499719, 499721, 499723,
       499725, 499729, 499731, 499733, 499735, 499737, 499741, 499743,
       499745, 499751, 499753, 499757, 499759, 499761, 499763, 499765,
       499767, 499769, 499771, 499773, 499775, 499777, 499779, 499781,
       499783, 499785, 499787, 499789])

In [77]:
normal_metabId = filtered_data[filtered_data['TN'] == 'Normal']['MetabID'].values
tumor_metabId = filtered_data[filtered_data['TN'] == 'Tumor']['MetabID'].values
normal_rnaId= filtered_data[filtered_data['TN'] == 'Normal']['RNAID'].values
tumor_rnaId = filtered_data[filtered_data['TN'] == 'Tumor']['RNAID'].values

In [79]:
import pandas as pd

combined_data = pd.DataFrame()

for column in metabolomic_data[common_elements].columns[1:]:
    df = pd.DataFrame({metabolomic_data['Unnamed: 0'].name: [column], **dict(zip(metabolomic_data['Unnamed: 0'], metabolomic_data[column]))})
    combined_data = pd.concat([combined_data, df], axis=0, ignore_index=True)

combined_data = pd.concat([combined_data['Unnamed: 0'], combined_data.drop(columns='Unnamed: 0')], axis=1)



,Unnamed: 0,1-arachidonoylglycerophosphoethanolamine*,1-arachidonoylglycerophosphoinositol*,1-linoleoylglycerophosphocholine,1-methylimidazoleacetate,1-methylnicotinamide,1-myristoylglycerophosphocholine,1-oleoylglycerol (1-monoolein),1-oleoylglycerophosphocholine,1-oleoylglycerophosphoethanolamine,...,urate,urea,uridine,ursodeoxycholate,valerylcarnitine,valine,xanthine,xanthosine,xylitol,xylulose
0,499529,32164.29,NaN,NaN,NaN,16319.61,NaN,NaN,NaN,NaN,...,NaN,53962.75,205765.49,NaN,NaN,4966602.9,42006.77,NaN,NaN,NaN
1,499533,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,46953.26,560535.90,116459.87,NaN,NaN,988931.8,40321.39,NaN,NaN,NaN
2,499535,72287.38,37584.38,NaN,NaN,119439.33,NaN,NaN,NaN,NaN,...,129572.95,793664.09,378524.63,NaN,16817.58,5746461.3,111113.47,69074.06,NaN,23675.09
3,499537,17454.59,16864.55,NaN,NaN,33167.55,NaN,NaN,NaN,NaN,...,128291.46,533508.79,230145.23,NaN,NaN,1722635.5,49325.50,NaN,NaN,NaN
4,499539,123149.33,NaN,61884.72,17584.41,57809.45,NaN,NaN,114391.93,NaN,...,189761.11,1279529.09,443456.69,NaN,71736.54,5547349.4,189579.74,28677.85,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,499781,NaN,NaN,NaN,29508.72,132603.92,NaN,81017.989,16423.80,NaN,...,164285.54,725892.90,578812.01,NaN,NaN,14667511.2,191270.18,61401.03,NaN,NaN
103,499783,57108.48,28504.03,NaN,NaN,71440.01,NaN,97146.180,558603.40,43873.59,...,158821.36,979630.25,419645.16,NaN,52358.77,12963713.0,283098.62,45147.44,NaN,NaN
104,499785,132447.15,75311.13,NaN,NaN,279805.79,NaN,21692.606,NaN,NaN,...,456463.36,9216881.17,880294.05,NaN,81993.73,8596174.7,95019.04,NaN,NaN,NaN
105,499787,34499.68,NaN,NaN,NaN,82356.41,NaN,NaN,23283.99,NaN,...,77267.21,480115.93,286030.98,NaN,NaN,4593273.6,70350.31,NaN,NaN,NaN


In [85]:
metabId_index_of_data = combined_data[['Unnamed: 0']]

In [101]:

for index, row in combined_data.iterrows():
    if(row['Unnamed: 0'].astype(int) in tumor_metabId.astype(int)):
        combined_data.at[index, 'Unnamed: 0'] = 'c'
    else: 
        combined_data.at[index, 'Unnamed: 0'] = 'healthy'

combined_data

,Unnamed: 0,1-arachidonoylglycerophosphoethanolamine*,1-arachidonoylglycerophosphoinositol*,1-linoleoylglycerophosphocholine,1-methylimidazoleacetate,1-methylnicotinamide,1-myristoylglycerophosphocholine,1-oleoylglycerol (1-monoolein),1-oleoylglycerophosphocholine,1-oleoylglycerophosphoethanolamine,...,urate,urea,uridine,ursodeoxycholate,valerylcarnitine,valine,xanthine,xanthosine,xylitol,xylulose
0,healthy,32164.29,NaN,NaN,NaN,16319.61,NaN,NaN,NaN,NaN,...,NaN,53962.75,205765.49,NaN,NaN,4966602.9,42006.77,NaN,NaN,NaN
1,healthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,46953.26,560535.90,116459.87,NaN,NaN,988931.8,40321.39,NaN,NaN,NaN
2,healthy,72287.38,37584.38,NaN,NaN,119439.33,NaN,NaN,NaN,NaN,...,129572.95,793664.09,378524.63,NaN,16817.58,5746461.3,111113.47,69074.06,NaN,23675.09
3,healthy,17454.59,16864.55,NaN,NaN,33167.55,NaN,NaN,NaN,NaN,...,128291.46,533508.79,230145.23,NaN,NaN,1722635.5,49325.50,NaN,NaN,NaN
4,healthy,123149.33,NaN,61884.72,17584.41,57809.45,NaN,NaN,114391.93,NaN,...,189761.11,1279529.09,443456.69,NaN,71736.54,5547349.4,189579.74,28677.85,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,c,NaN,NaN,NaN,29508.72,132603.92,NaN,81017.989,16423.80,NaN,...,164285.54,725892.90,578812.01,NaN,NaN,14667511.2,191270.18,61401.03,NaN,NaN
103,c,57108.48,28504.03,NaN,NaN,71440.01,NaN,97146.180,558603.40,43873.59,...,158821.36,979630.25,419645.16,NaN,52358.77,12963713.0,283098.62,45147.44,NaN,NaN
104,c,132447.15,75311.13,NaN,NaN,279805.79,NaN,21692.606,NaN,NaN,...,456463.36,9216881.17,880294.05,NaN,81993.73,8596174.7,95019.04,NaN,NaN,NaN
105,c,34499.68,NaN,NaN,NaN,82356.41,NaN,NaN,23283.99,NaN,...,77267.21,480115.93,286030.98,NaN,NaN,4593273.6,70350.31,NaN,NaN,NaN


In [125]:
factors = combined_data.iloc[:, 0:1]
factors

,Unnamed: 0
0,healthy
1,healthy
2,healthy
3,healthy
4,healthy
...,...
102,c
103,c
104,c
105,c


In [105]:
merged_df = pd.concat([metabId_index_of_data, factors], axis=1)

merged_df.to_csv('./datasets/cancer atlas/metabId_and_factors.csv')

In [63]:
combined_data = combined_data.rename(columns={'Unnamed: 0': 'Factors'})

In [65]:
combined_data.to_csv('datasets/cancer atlas/processed_metabolomics.csv', index=False)

In [129]:
metabId_and_rnaId_mapping

for index, row in metabId_and_rnaId_mapping.iterrows():
    old_path = './updated_recon3d_samples/updated_Recon3D_' + str(row['RNAID']) + '.json'
    new_path = './updated_recon3d_samples/updated_Recon3D_' + str(row['MetabID']) + '.json'

    if os.path.exists(old_path):
        os.rename(old_path, new_path)



In [133]:
indexNumber = 0
for index, row in merged_df.iterrows():
    old_path = './updated_recon3d_samples/updated_Recon3D_' + str(row['Unnamed: 0'][0]) + '.json'
    new_path = './updated_recon3d_samples/updated_Recon3D_' + str(indexNumber) + '.json'
    if os.path.exists(old_path):
        indexNumber += 1
        os.rename(old_path, new_path)


In [111]:
merged_df.iloc[:, 1:]

,Unnamed: 0
0,healthy
1,healthy
2,healthy
3,healthy
4,healthy
...,...
102,c
103,c
104,c
105,c
